In [28]:
import torch
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [30]:
with open("C:/Users/User/Documents/Machine Learning/social listening project/tf_idf.csv", encoding="utf-8") as f:
    reader = csv.reader(f)
    data_list = [row for row in reader]
    data_list = np.asarray(data_list[0:5500],dtype=np.float64)
    data_list = np.concatenate((np.ones((data_list.shape[0],1)), data_list), axis = 1)
f.close()

In [31]:
with open("C:/Users/User/Documents/Machine Learning/social listening project/Data dùng được.csv", encoding="utf-8") as f:
    reader = csv.reader(f)
    label_list = [row for row in reader]
    label_list = label_list[0:5500]
    # 0 = Ukraina and 1 = Nga
    for i in range(0,len(label_list)):
        if label_list[i][1]=="U":
            label_list[i]=0
        else: 
            label_list[i]=1 
f.close()

In [ ]:
X_train, X_test,X_val, y_train, y_test,y_val = train_test_split(data_list, label_list, train_size=7/10,test_size=2/10,val_size=1/10, random_state=0)
X_train = torch.tensor(X_train,dtype=torch.float64)
X_test = torch.tensor(X_test,dtype=torch.float64)
X_val = torch.tensor(X_val,dtype=torch.float64)
y_train = torch.tensor(y_train,dtype=torch.float64)
y_test = torch.tensor(y_test,dtype=torch.float64)
y_val = torch.tensor(y_val,dtype=torch.float64)

In [33]:
def sigmoid(s):
    return (1/(1 + 1e-6 + torch.exp(-s)))

In [34]:
def loss_function(w,X_train,y_train):
    z = sigmoid(torch.matmul(w,torch.transpose(X_train,0,1)))
    loss = torch.sum(-(y_train*torch.log(z)+(1-y_train)*torch.log(1-z)))
    return loss

In [91]:

def logistic_sigmoid_regression(X_train, y_train, w_init, eta,X_val, y_val,tol = 1e-5, max_count = 300000):
    w = [w_init]    # vector trọng số
    #it = 0
    N = X_train.shape[0]  # số đối tượng
    count = 0   # đến số vong lặp
    check_w_after = 20  # check lại w sau mỗi 20 lần lặp
    loss_list = [0]
    count_list = []
    while count<max_count:
        # mix data 
        mix_id = torch.randperm(N)
        # cập nhật w sau mỗi vòng lặp
        z = sigmoid(torch.matmul(w[-1],torch.transpose(X_train,0,1)))
        for i in mix_id:
            # xi = X_train[i]
            # yi = y_train[i]
            # zi = sigmoid(torch.dot(w[-1],xi))
            w_new = w[-1] + eta*(y_train[i] - z[i])*X_train[i]
            count += 1
            # điều kiện dừng vòng lặp
            if count%check_w_after == 0:     
                loss_list.append(loss_function(w_new,X_val,y_val)) 
                count_list.append(count)
                print(count,loss_list[-1])
                if abs(loss_list[-1]-loss_list[-2])<tol:
                    return w[-1],loss_list,count_list
            w.append(w_new)
    return w[-1],loss_list,count_list

In [ ]:
eta = .002   # learning rate = 0.01
d = X_train.shape[1]
w_init = torch.randn(d,dtype=torch.float64)  # lấy random một vector w
w, loss_list, count_list = logistic_sigmoid_regression(X_train, y_train, w_init, eta,X_val, y_val)
w = w.reshape((1,-1))

In [ ]:
plt.plot(count_list,loss_list[1:])

In [ ]:
predict = sigmoid(torch.matmul(w,torch.transpose(X_test,0,1)))
count = 0
for i in range(0,len(predict[0])):
    x = predict[0][i]
    y = y_test[i]
    if (x + y >=1.5) or (x + y <0.5):
        count+=1
    #print("%.4f" %predict[0][i],y_test[i])
print(count)
print(len(y_test))
print(count/len(y_test))

In [ ]:
for i in range(0,len(y_test)):
    print(y_test[i].item(),"%.2f" % predict[0][i].item())